In [ ]:
import pandas as pd

def klines(symbol, interval, start):
    params = {
        "symbol" : symbol,
        "interval" : interval,
        "startTime" : start
    }
    url = urljoin(base_url, endppoint)
    r = requests.get(url, params=params)
    code = r.status_code
    print("code : ",code)
    if code == 200:
        return r.json()
    

X = klines("BTCUSDT", interval="1d", start="2023")

col = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'num_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
df = pd.DataFrame(X, columns = col)
df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
df

In [ ]:
from datetime import datetime
import time

import requests
import hmac
import hashlib
from urllib.parse import urljoin, urlencode

import concurrent.futures

from dataEngine.extract import Binance


%load_ext autoreload
%autoreload 2

"GET /fapi/v1/continuousKlines"

base_url = "https://testnet.binancefuture.com"
endppoint = "/fapi/v1/klines"


def klines(symbol, interval, start):
    params = {
        "symbol" : symbol,
        "interval" : interval,
        "startTime" : start
    }
    url = urljoin(base_url, endppoint)
    r = requests.get(url, params=params)
    code = r.status_code
    print("code : ",code)
    if code == 200:
        return r.json()



all_data = {}

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(klines, symbol, interval, start) for symbol in symbols]
    
    for future in concurrent.futures.as_completed(futures):
        symbol, data = future.result()
        all_data[symbol] = data

if all_data:
    for symbol, data in all_data.items():
        print(f" symbol {symbol} : {data}")
 



In [ ]:
from TCed.agent import Agent
from TCed.globaleAgent import GlobalAgent
from TCed.env import Env

import multiprocessing

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

config = {"capital" : 100, "size" : 45, "interval" : "1d",
          "symbols" : ["BTC", "ETH", "SOL", "DASH", "MATIC",
                       "TWT", "GALA", "EGLD", "XMR"],
          "start" : "2022", "end" : "2023"
          }


env = Env(**config)

master = multiprocessing.Queue()
agents = multiprocessing.Queue()

ENV = multiprocessing.Queue()
ENV.put(env)

communication_lock = multiprocessing.Condition()

In [ ]:
from dataEngine.data import connect_db

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


SYMBOLS = ["DASH", "ETH", "XMR"]
CAPITAL = 100
START = "2023"
END = "2023"

datas = []

db = connect_db(name = "database", interval = "1d")

def gen_data(symbol, size = 10):
    i = 0
    data = db.get_data(symbol, start = START, end = END)
    while  True:
        batch = data.iloc[i : i + size]
        if len(batch) == size:
            yield batch
        else:
            break
        i += 1

x = gen_data("BTC")
x

In [ ]:
next(x)

# GenData

In [ ]:
import zmq
from IPython.display import clear_output
from dataEngine.data import connect_db

In [ ]:
SYMBOLS = ["DASH", "TWT"]
START = "2023-01"
END = "2023-01"

db = connect_db(name = "database", interval = "1d")
data = db.get_data(SYMBOLS[0], start = START, end = END)

In [ ]:
class Server:
    
    def __init__(self, symbols, start, end):
        self.symbols = symbols
        self.start = start 
        self.end = end
        
        self.get_data()
        self.create()
        
    
    def get_data(self):
        self.data = {}
        for symbol in self.symbols:
            data = db.get_data(symbol, start = self.start, end = self.end)
            self.data[symbol] = data
        
        
    def create(self):
        context = zmq.Context()
        self.socket = context.socket(zmq.PUB)
        self.socket.bind('tcp://127.0.0.1:8080')
        
    
    def line(self, symbol, data, bar):
        try:
            date = data.index[bar].timestamp()
            #date = date.timestamp()
            msg = '{} {} {} {} {} {} {}'.format(symbol, date,
                                            data.iloc[bar]['open'], data.iloc[bar]['high'],
                                            data.iloc[bar]['low'], data.iloc[bar]['close'],
                                            data.iloc[bar]['volume'])
        except:
            msg = "-"
            
        print(msg)
        self.socket.send_string(msg)
    
    
    def run(self):
        bar = 0
        while True:
            clear_output(True)
            for symbol in self.symbols:
                self.line(symbol, self.data.get(symbol), bar)
        
        bar += 1
        print("bar : ",bar)
        

In [ ]:
SYMBOLS = ["BTC", "ETH"]
START = "2023-01"
END = "2023-01"


server = Server(symbols = SYMBOLS, start=START, end=END)
server.run()

In [4]:
dic = {"a" : 3, "b" : 5}
dic.pop("a")

3

In [7]:
dic.pop("r", None)

# GlobalAgent

In [ ]:
import random
import multiprocessing


class GlobalAgent(multiprocessing.Process):
    
    def __init__(self, activeAgent = "", communication_lock="", agents="", master=""):
        multiprocessing.Process.__init__(self)
        self.activeAgent = activeAgent
        self.n_agent = 2
        
        self.agents = agents
        self.master = master
        
        self.communication_lock = communication_lock
                        
            
    def run(self):
        while True:
            
            with self.communication_lock:
                print(f" agents size {self.agents.qsize()}")
                self.communication_lock.wait_for(lambda : self.agents.qsize() == self.n_agent)
                print(f" agents size {self.agents.qsize()}")
                print("go the ----> ")
                
                for i in range(self.n_agent):
                    msg = self.agents.get()
                
                agent_id = random.randint(1, 4)
                self.activeAgent.value = agent_id
                self.master.put(msg)
                
                self.communication_lock.notify_all()
                


In [ ]:
import threading

def ma_fonction(arg1, arg2):
    print(f"Thread exécutant avec les arguments : {arg1}, {arg2}")

# Créez un thread avec des arguments
thread = threading.Thread(target=ma_fonction, args=("valeur1", "valeur2"))

# Démarrez le thread
thread.start()

# Attendez que le thread se termine
thread.join()


In [ ]:
import threading

verrou = threading.Lock()
compteur = 0

def incrementer_compteur(name):
    global compteur
    for i in range(5):
        with verrou:
            print(f"agent : {name}")
            compteur += 1

params1 = {"name" : 1}
params2 = {"name" : 2}
params3 = {"name" : 3}

# Créez deux threads pour incrémenter le compteur
thread1 = threading.Thread(target=incrementer_compteur, kwargs=params1)
thread2 = threading.Thread(target=incrementer_compteur, kwargs=params2)
thread3 = threading.Thread(target=incrementer_compteur, kwargs=params3)

thread1.start()
thread2.start()
thread3.start()

thread1.join()
thread2.join()
thread3.join()

print("Compteur final :", compteur)


In [ ]:
import threading
import time

def attendre_evenement():
    print("Thread 1 attend l'événement.")
    evenement.wait()
    print("Thread 1 a reçu l'événement.")

def declencher_evenement():
    print("Thread 2 déclenche l'événement.")
    time.sleep(3)  # Attendez un moment pour illustrer que le thread 1 attend
    evenement.set()

evenement = threading.Event()

thread1 = threading.Thread(target=attendre_evenement)
thread2 = threading.Thread(target=declencher_evenement)

thread1.start()
thread2.start()

thread1.join()
thread2.join()

print("Les threads ont terminé.")


In [3]:
import threading

def imprimer_pairs():
    for i in range(0, 10, 2):
        with condition:
            while not pair[0]:
                condition.wait()
            print(i)
            pair[0] = False
            condition.notify()

def imprimer_impairs():
    for i in range(1, 10, 2):
        with condition:
            while pair[0]:
                condition.wait()
            print(i)
            pair[0] = True
            condition.notify()

condition = threading.Condition()
pair = [True]

thread1 = threading.Thread(target=imprimer_pairs)
thread2 = threading.Thread(target=imprimer_impairs)

thread1.start()
thread2.start()

thread1.join()
thread2.join()

print("Les threads ont terminé.")


0
1
2
3
4
5
6
7
8
9
Les threads ont terminé.


In [ ]:
import threading

class RessourcePartagee:
    def __init__(self):
        self.lock = threading.Lock()
        self.condition = threading.Condition(lock=self.lock)
        self.contenu = None

    def set_contenu(self, contenu):
        with self.lock:
            self.contenu = contenu
            self.condition.notify()

    def get_contenu(self):
        with self.lock:
            while self.contenu is None:
                self.condition.wait()
            return self.contenu

def producteur(ressource):
    ressource.set_contenu("Données produites")

def consommateur(ressource):
    data = ressource.get_contenu()
    print("Données consommées:", data)

ressource_partagee = RessourcePartagee()

thread_producteur = threading.Thread(target=producteur, args=(ressource_partagee,))
thread_consommateur = threading.Thread(target=consommateur, args=(ressource_partagee,))

thread_producteur.start()
thread_consommateur.start()

thread_producteur.join()
thread_consommateur.join()

print("Les threads ont terminé.")


In [ ]:
import threading

# Créez un sémaphore avec 2 jetons
semaphore = threading.Semaphore(2)

def utiliser_ressource(id):
    with semaphore:
        print(f"Thread {id} utilise la ressource")
        # Simulez l'utilisation de la ressource
        for _ in range(3):
            pass
        print(f"Thread {id} a terminé d'utiliser la ressource")

# Créez plusieurs threads qui utilisent la ressource
threads = []
for i in range(5):
    thread = threading.Thread(target=utiliser_ressource, args=(i,))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()


In [ ]:
import threading

class Agent(threading.Thread):
    def __init__(self, name, condition):
        super().__init__()
        self.name = name
        self.condition = condition

    def run(self):
        with self.condition:
            print(f"{self.name} en attente...")
            self.condition.wait()
            print(f"{self.name} se réveille et agit.")

def main():
    condition = threading.Condition()
    agents = []

    # Créez 4 agents
    for i in range(1, 5):
        agent = Agent(f"Agent {i}", condition)
        agents.append(agent)
        agent.start()

    # Attendre un moment
    import time
    time.sleep(2)

    with condition:
        print("Réveiller tous les agents...")
        condition.notify_all()

    # Attendre que tous les agents terminent
    for agent in agents:
        agent.join()

if __name__ == "__main__":
    main()


### je veux 3 agent et un master en utilisant les Condition

In [1]:
import threading

class Master:
    def __init__(self, condition):
        self.condition = condition

    def run(self):
        for _ in range(10):
            with self.condition:
                print("Maître : J'autorise les agents à travailler.")
                self.condition.notify_all()
                print("Autorisation accordee a tous les agents .. ")
                print("Master : Wait for agent response ... ")
                self.condition.wait()
                print("Master : I get data ")

class Agent:
    def __init__(self, name, condition):
        self.name = name
        self.condition = condition

    def run(self):
        for _ in range(10):
            with self.condition:
                print(f"Agent {self.name} : J'attends l'autorisation du maître.")
                self.condition.wait()
                print(f"Agent {self.name} : Travail en cours.")
                self.condition.notify()
                self.condition.wait()
                print(f"Agent {self.name} : Travail terminé.")
                self.condition.notify()

# Créez une Condition pour coordonner les agents et le maître
condition = threading.Condition()

# Créez le maître et les agents
master = Master(condition)
agents = [Agent(str(i), condition) for i in range(1, 4)]

# Créez des threads pour le maître et les agents
master_thread = threading.Thread(target=master.run)
agent_threads = [threading.Thread(target=agent.run) for agent in agents]

# Démarrez les threads
master_thread.start()
for thread in agent_threads:
    thread.start()

# Attendez que tous les threads se terminent
master_thread.join()
for thread in agent_threads:
    thread.join()

print("Tous les agents et le maître ont terminé.")


Maître : J'autorise les agents à travailler.
Autorisation accordee a tous les agents .. 
Master : Wait for agent response ... 
Agent 1 : J'attends l'autorisation du maître.
Agent 2 : J'attends l'autorisation du maître.
Agent 3 : J'attends l'autorisation du maître.


### je veux 3 agent et un master en utilisant les Condition. Les agents ont des parametres differents

In [1]:
import threading

class Master:
    def __init__(self, condition):
        self.condition = condition
        self.work_to_be_done = []

    def assign_work(self, work):
        with self.condition:
            self.work_to_be_done.append(work)
            self.condition.notify_all()

    def run(self):
        for _ in range(10):
            with self.condition:
                while not self.work_to_be_done:
                    print("Maître : J'attends que les agents aient du travail.")
                    self.condition.wait()
                #print("Work : ",self.work_to_be_done)
                work = self.work_to_be_done.pop(0)
                print(f"Maître : Autorisation d'exécuter le travail pour l'agent {work['agent_id']} ({work['param']}).")
                self.condition.notify_all()


class Agent:
    def __init__(self, agent_id, condition):
        self.agent_id = agent_id
        self.condition = condition

    def run(self, param):
        for _ in range(10):
            with self.condition:
                self.condition.wait()
                print(f"Agent {self.agent_id} : Travail recu de la part du master")
                print(f"Agent {self.agent_id} : Travail en cours avec paramètre {param}.")
                self.condition.notify()
                print("**********************")
                self.condition.wait()
                print(f"Agent {self.agent_id} : Travail terminé.")
                self.condition.notify()

# Créez une Condition pour coordonner les agents et le maître
condition = threading.Condition()

# Créez le maître et les agents
master = Master(condition)
agents = [Agent(i, condition) for i in range(1, 4)]

# Créez des threads pour le maître et les agents
master_thread = threading.Thread(target=master.run)
agent_threads = [threading.Thread(target=agent.run, args=(f"paramètre-{i}",)) for i, agent in enumerate(agents, start=1)]

# Démarrez les threads
master_thread.start()
for thread in agent_threads:
    thread.start()

# Assignez du travail aux agents
for i, agent in enumerate(agents, start=1):
    work = {"agent_id": i, "param": f"paramètre-{i}"}
    master.assign_work(work)

# Attendez que tous les threads se terminent
master_thread.join()
for thread in agent_threads:
    thread.join()

print("Tous les agents et le maître ont terminé.")


Maître : J'attends que les agents aient du travail.
Maître : Autorisation d'exécuter le travail pour l'agent 1 (paramètre-1).
Maître : Autorisation d'exécuter le travail pour l'agent 2 (paramètre-2).
Maître : Autorisation d'exécuter le travail pour l'agent 3 (paramètre-3).
Maître : J'attends que les agents aient du travail.
Agent 1 : Travail recu de la part du master
Agent 1 : Travail en cours avec paramètre paramètre-1.
**********************
Agent 2 : Travail recu de la part du master
Agent 2 : Travail en cours avec paramètre paramètre-2.
**********************
Agent 3 : Travail recu de la part du master
Agent 3 : Travail en cours avec paramètre paramètre-3.
**********************
Maître : J'attends que les agents aient du travail.
Agent 1 : Travail terminé.
Agent 2 : Travail terminé.
Agent 3 : Travail terminé.
Maître : J'attends que les agents aient du travail.
Agent 1 : Travail recu de la part du master
Agent 1 : Travail en cours avec paramètre paramètre-1.
**********************
A

### Le master doit pouvoir identifier les agents

In [ ]:
import threading

class Master:
    def __init__(self, condition):
        self.condition = condition
        self.work_to_be_done = {}

    def assign_work(self, agent_id, param):
        with self.condition:
            self.work_to_be_done[agent_id] = param
            self.condition.notify_all()

    def run(self):
        for _ in range(10):
            with self.condition:
                while not self.work_to_be_done:
                    print("Maître : J'attends que les agents aient du travail.")
                    self.condition.wait()
                agent_id, param = self.work_to_be_done.popitem()
                print(f"Maître : Autorisation d'exécuter le travail pour l'agent {agent_id} ({param}).")
                self.condition.notify_all()

class Agent:
    def __init__(self, agent_id, condition):
        self.agent_id = agent_id
        self.condition = condition

    def run(self):
        for _ in range(10):
            with self.condition:
                self.condition.wait()
                print(f"Agent {self.agent_id} : Travail en cours avec paramètre {param}.")
                self.condition.notify()
                self.condition.wait()
                print(f"Agent {self.agent_id} : Travail terminé.")
                self.condition.notify()

# Créez une Condition pour coordonner les agents et le maître
condition = threading.Condition()

# Créez le maître et les agents
master = Master(condition)
agents = [Agent(i, condition) for i in range(1, 4)]

# Créez des threads pour le maître et les agents
master_thread = threading.Thread(target=master.run)
agent_threads = [threading.Thread(target=agent.run) for agent in agents]

# Démarrez les threads
master_thread.start()
for thread in agent_threads:
    thread.start()

# Assignez du travail aux agents en spécifiant leur identifiant
for agent in agents:
    work = f"paramètre-{agent.agent_id}"
    master.assign_work(agent.agent_id, work)

# Attendez que tous les threads se terminent
master_thread.join()
for thread in agent_threads:
    thread.join()

print("Tous les agents et le maître ont terminé.")


### Le master doit pouvoir identifier les agents et envoyer un message a un agent specifique

In [ ]:
import threading

class Master:
    def __init__(self, condition):
        self.condition = condition
        self.work_to_be_done = {}
        self.agents = {}

    def register_agent(self, agent_id, agent):
        self.agents[agent_id] = agent

    def assign_work(self, agent_id, param):
        with self.condition:
            self.work_to_be_done[agent_id] = param
            self.condition.notify_all()

    def send_message(self, agent_id, message):
        if agent_id in self.agents:
            agent = self.agents[agent_id]
            agent.receive_message(message)
        else:
            print(f"Maître : Agent {agent_id} n'existe pas.")

    def run(self):
        for _ in range(10):
            with self.condition:
                while not self.work_to_be_done:
                    print("Maître : J'attends que les agents aient du travail.")
                    self.condition.wait()
                agent_id, param = self.work_to_be_done.popitem()
                print(f"Maître : Autorisation d'exécuter le travail pour l'agent {agent_id} ({param}).")
                self.condition.notify_all()

class Agent:
    def __init__(self, agent_id, condition):
        self.agent_id = agent_id
        self.condition = condition

    def receive_message(self, message):
        print(f"Agent {self.agent_id} a reçu un message : {message}")

    def run(self):
        for _ in range(10):
            with self.condition:
                self.condition.wait()
                print(f"Agent {self.agent_id} : Travail en cours avec paramètre {param}.")
                self.condition.notify()
                self.condition.wait()
                print(f"Agent {self.agent_id} : Travail terminé.")
                self.condition.notify()

# Créez une Condition pour coordonner les agents et le maître
condition = threading.Condition()

# Créez le maître et les agents
master = Master(condition)
agents = [Agent(i, condition) for i in range(1, 4)]

# Enregistrez les agents auprès du maître
for agent in agents:
    master.register_agent(agent.agent_id, agent)

# Créez des threads pour le maître et les agents
master_thread = threading.Thread(target=master.run)
agent_threads = [threading.Thread(target=agent.run) for agent in agents]

# Démarrez les threads
master_thread.start()
for thread in agent_threads:
    thread.start()

# Assignez du travail aux agents en spécifiant leur identifiant
for agent in agents:
    work = f"paramètre-{agent.agent_id}"
    master.assign_work(agent.agent_id, work)

# Envoyez un message à un agent spécifique
master.send_message(2, "Message pour agent 2")

# Attendez que tous les threads se terminent
master_thread.join()
for thread in agent_threads:
    thread.join()

print("Tous les agents et le maître ont terminé.")


In [ ]:
""

In [ ]:
a = []
not a

In [ ]:
import threading
import random
import time


def action():
    print("***** La barrier a ete franchi ******* ")

# Fonction que les threads exécuteront
def worker(barrier, stage):
    for _ in range(2):
        print(f"---> step {_} _ thread waiting : {barrier.n_waiting}")
        print(f"--------- Thread {threading.current_thread().name} : ----------")
        print(f"step {_} _ Thread {threading.current_thread().name} : Début de l'étape {stage}")
        for i in range(3):
            print(f"step {_} _ Thread {threading.current_thread().name} : Travail en cours à l'étape {stage}, itération {i}")
        #time.sleep(random.randint(1, 3))
        print(f"step {_} _ Thread {threading.current_thread().name} : Fin de l'étape {stage}")
        print(f"step {_} _ thread waiting : {barrier.n_waiting}")
        print(f"step {_} _ Thread {threading.current_thread().name} : Travail terminé.")
        print(f" ------Fin Step {_}--------- \n")
        barrier.wait()
        
        
        
# Créez une barrière pour 3 threads
barrier = threading.Barrier(3, action=action)

# Créez trois threads
threads = [threading.Thread(name=1, target=worker, args=(barrier, 1)),
           threading.Thread(name=2, target=worker, args=(barrier, 2)),
           threading.Thread(name=3, target=worker, args=(barrier, 3))]

# Démarrez les threads
for thread in threads:
    thread.start()

# Attendez que tous les threads se terminent
for thread in threads:
    thread.join()

print("Tous les threads ont terminé.")


In [ ]:
import threading

def action():
    print("***** La barrier a ete franchi ******* ")
    
    
# Classe pour les agents
class Agent(threading.Thread):
    def __init__(self, agent_id, event_start, event_finish, iterations):
        super().__init__()
        self.agent_id = agent_id
        self.event_start = event_start
        self.event_finish = event_finish
        self.iterations = iterations

    def run(self):
        for i in range(self.iterations):
            print(f"Agent {self.agent_id} : Itération {i}")
            self.event_start.wait()  # Attend le signal de démarrage du maître
            print(f"Agent {self.agent_id} : Début du travail à l'itération {i}")
            # Effectuer le travail de l'agent
            print(f"Agent {self.agent_id} : Travail terminé à l'itération {i}")
            self.event_finish.set()  # Signale la fin de l'itération

# Classe pour le maître
class Master(threading.Thread):
    def __init__(self, event_start, event_finish, iterations):
        super().__init__()
        self.event_start = event_start
        self.event_finish = event_finish
        self.iterations = iterations

    def run(self):
        print("Maître : Début du travail")
        for i in range(self.iterations):
            # Effectuer le travail du maître à chaque itération
            print("Maître : Travail du maître à l'itération", i)
            self.event_start.set()  # Signale le début de l'itération aux agents
            self.event_finish.wait()  # Attend que tous les agents aient terminé l'itération
            self.event_finish.clear()  # Réinitialise l'événement de fin pour la prochaine itération
        print("Maître : Travail terminé")

# Créez des objets Event pour le démarrage et la fin de chaque itération
event_start = threading.Event()
event_finish = threading.Event()

# Créez trois threads d'agent
agent_threads = [Agent(i, event_start, event_finish, 3) for i in range(1, 4)]

# Créez un thread de maître
master_thread = Master(event_start, event_finish, 3)

# Démarrez les threads d'agent
for thread in agent_threads:
    thread.start()

# Démarrez le thread de maître
master_thread.start()

# Attendez que le maître et tous les agents se terminent
master_thread.join()
for thread in agent_threads:
    thread.join()

print("Tous les agents et le maître ont terminé.")


In [ ]:
from contextlib import contextmanager

@contextmanager
def mon_context_manager():
    # Code d'initialisation
    print("Entrée dans le contexte")
    try:
        yield  # C'est l'équivalent de la méthode __enter__()
    finally:
        # Code de nettoyage
        print("Sortie du contexte")

# Utilisation du context manager
with mon_context_manager():
    # Code à l'intérieur du contexte
    print("À l'intérieur du contexte")
